# MA934  Numerical Methods - Workbook 2

In [33]:
using Plots
using Distributions
Plots.plotly()
include("kvpair.jl")
include("llist.jl")
include("ftree.jl")
include("functions.jl")

LoadError: LoadError: invalid redefinition of constant LList
while loading /home/edoardo/Downloads/Complexity/Numerical Methods/Assignements/Assignement 2/llist.jl, in expression starting on line 2

In [3]:
#### GLOBAL VARS ####
sizes = 2.^(1:10)
 # Seed for random number generator. 
# You should choose your own seed so that everyone does not end up with the same random sequences
seed = 666 

# Initialise the random number generator
rng = MersenneTwister(seed)

MersenneTwister(UInt32[0x0000029a],Base.dSFMT.DSFMT_state(Int32[-1829720698,1073617959,-209863392,1073342874,851925708,1073663428,929754575,1073410908,-1469009651,1073283821  …  -1656730289,1072845189,1382744295,1073272649,-1940720456,-2031062331,1988518479,1214015520,382,0]),[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],382)

## Question 1: Linear search using a linked list

The file KVPair.jl defines a simple data structure to represent an (Int64, Float64) key-value pair. 

The file LList.jl defines a linked list that stores a KVPair at each node. Many languages have a keyword NULL. A pointer to any object can take the value NULL to indicate that it doesn't point to anything. NULL is often used to denote the end of a list, the leaves of a tree etc. Julia does not support NULL pointers. This causes a difficulty in implementing recursive data structures. Instead Julia provides a parametric data type Nullable{T} to represent missing values. It is possible to use Nullable types to define recursive data structures although some of the resulting definitions are a bit clumsy. I'm not sure that this is the canonical way to implement structural recursion in Julia. 

The following points are important:

* if a variable has type Nullable{T} then it can either contain a value of type T or nothing (ie a missing value). 
* the function isnull(x) checks whether a nullable type x has a value or not. 
* the function get(x) returns the actual value (of type T) contained in a nullable type x.

Much more information can be found at https://docs.julialang.org/en/stable/manual/types/

The function 

> buildLList(dataArray::Array{KVPair, 1})

takes an array of KVPair objects as input and returns an LList containing these KVPairs. This is illustrated in the code below.

1. Write a recursive function that traverses the list and prints out the key-value pairs stored in it. Check that your function works.
2. Write a function search(list::Nullable{LList}, k::Int64) that searches an LList for the key k and returns the corresponding KVPair if it is present and a Nullable{KVPair} otherwise. Verify that your function works.
3. Use Julia's @timed macro to measure how the typical computational cost of your search() function grows with the length of the list. 

In [11]:
list = generate_llist(5)

list_trasverser(list)

# Test SPEEEEEED
repetitions = 30
sizes = 2.^(1:12)
times = Array(Float64, 0)
repetition_times = zeros(repetitions)

for size in sizes
    for i in 1:repetitions
        # Generate list 
        list = generate_llist(size)
        # Get the time for the search, given random key to search
        tmp = @timed search_list(list, rand(1:size))
        # Add to times array
        repetition_times[i] = tmp[2]
    end
    # Save the averages of executions for a given size
    push!( times, mean(repetition_times))
end

@show times

Plots.plot(2.^(1:12), times)

node.data = KVPair(1,0.600172671143351)
node.data = KVPair(2,0.7480518123051316)
node.data = KVPair(3,0.25716989482076613)
node.data = KVPair(4,0.8149809648855131)
node.data = KVPair(5,0.2897441989895193)
times = [2.42067e-7,1.96633e-7,1.696e-7,2.002e-7,2.70433e-7,4.61733e-7,9.28667e-7,1.9723e-6,4.81717e-6,6.5116e-6,1.79789e-5,3.78446e-5]


## Question 2: Interval membership

The problem of interval membership is the following: given a set of contiguous intervals, 

$\left\{[x_0, x_1), [x_1, x_2), \ldots, [x_{n-2}, x_{n-1}), [x_{n-1}, x_n)\right\}$

spanning the interval $[x_1, x_n)$ and given a random number $x \in [x_1, x_n)$, determine the interval in which $x$ lies. The standard numerical algorithm for stochastic simulation of continuous-time Markov processes (eg birth-death process, contact process, SIR model etc) requires solving an interval membership problem at each time step. It is therefore important to be able to solve it efficiently. Note, we cannot assume that all intervals are the same length.

We can solve the interval membership problem with $n$ interval by a variant of linear search in $O(n)$ time. We will demonstrate below that it can be solved in $O(\log\, n)$ time using a variant of a data structure known as a Fenwick tree.

**1)** Use Julia's random number generator to generate $n$ random interval lengths, $y_1, y_2, \ldots, y_n$ between 0 and 1 as shown: 

The corresponding interval membership problem is constructed from the partial sums:
$$x_i = \sum_{j=1}^i y_j, \ \ \ \ \ i=1,2,\ldots n.$$ 
Using the objects defined in KVPair.jl and LList.jl, create an array of key-value pairs, $(i, x_i)$, associating each interval with the corresponding partial sum and store this array in a linked list. Print the list for a small value of $n$to verify that it works.

**2)** Modify the search function you wrote above to write a recursive function

$$\text{intervalmembership(list::Nullable{LList}, x::Float64)}$$


## I called it 
> intervalmembership_brute 

## So that I could have both functions defined in the same file without overwriting

that takes the LList containing the list of partial sums and a random Float64 in the range $[0, x_n]$ as inputs and returns the KVPair corresponding to the interval in which $x$ lies. Verify that it works for small values of $n$ and use Julia's @timed macro to measure how the typical computational cost grows with $n$. 



In [26]:
# Setting parameters
repetitions = 100
times_brute = Array(Float64, 0)
repetition_times = zeros(repetitions)

for size in sizes
    ### Loop over sizes
    for i in 1:repetitions
        ### Loop over repetitions
        
        # Generate intervals
        llist, cumsum = generate_cumulative_llist(size, rng)

        # Get the time for the search, given random value in range
        to_search = rand()*cumsum[end]
        tmp = @timed intervalmembership_brute(llist, to_search)
                
        # Add to times array
        repetition_times[i] = tmp[2]
    end
    # Save the averages of executions for a given size
    push!( times_brute, mean(repetition_times))
end

In [27]:
x = sizes
Plots.plot(x, times_brute, label="Empirical")

**3)** The file FTree.jl defines a data structure implementing a variant of a Fenwick tree that can solve the interval membership problem as described in the lectures/notes. The function 

$$\text{buildFTree(T::Nullable{FTree}, dataArray::Array{KVPair, 1})}$$

takes the array of KVPairs containing the interval lengths as input, recursively constructs the tree and returns the FTree containing the correct key-value pairs (note a key of -1 is assigned to all non-leaf nodes).

In [28]:
# Set up variables
# Note: sizes defined earlier (need to be the same for the two problems)
repetitions = 1000
times = Array(Float64, 0)
repetition_times = zeros(repetitions)

for size in sizes
    ### Loop over sizes
    for i in 1:repetitions
        ### Loop over repetitions
        
        # Generate intervals
        values,cumsum,rng = generate_intervals(size, rng)
        
        # Set up tree
        T = Nullable{FTree}(FTree(KVPair(0,0.0)))
        T = buildFTree(T, values);
        
        # Get the time for the search, given random value in range
        to_search = rand()*cumsum[end]
        tmp = @timed intervalmembership(T, to_search)
        
        # Add to times array
        repetition_times[i] = tmp[2]
    end
    # Save the averages of executions for a given size
    push!( times, mean(repetition_times))
end


Plots.plot(times)

In [36]:
x = 2.^(1:8)
Plots.plot(x, times, label="Tree Search")
Plots.plot!(x, times_brute, label="List Search")
Plots.plot!(x, log.(x)*times[1], label="Theoretical")

Write a recursive function

$$\text{intervalmembership(FT::Nullable{FTree}, x::Float64)}$$

that takes the FTree containing the list of partial sums and a random Float64 in the range $[0, x_n]$ as 
inputs and returns the KVPair corresponding to the interval in which $x$ lies. Verify that it works for small values of $n$ and use Julia's @timed macro to compare how the typical computational compares to the above linear search version. Note it can sometimes be difficult to directly measure $O(\log\, n)$ growth in computation time since the problem size needs to become very large to register appreciable run times.

## Question 3: Gillespie algorithm for heterogeneous diffusion

The Gillespie algorithm is a dynamic Monte Carlo algorithm that generates statistically correct realisations of a continous time Markov process. See https://en.wikipedia.org/wiki/Gillespie_algorithm It is used heavily in stochastic simulation of chemical and biological reaction networks and population dynamics. In this question we use it to simulate one-dimensional diffusion of heterogeneous particles whose diffusion constants are independently drawn from an exponential distribution.

The basic idea of the algorithm is as follows
1. Create a list $\left\{(i, r_i), i=1,\ldots M\right\}$,  of all possible transitions (events), $i$, that can occur in the system and the corresponding transition rates, $r_i$.
2. Calculate the total rate
$$R = \sum_{i=1}^M r_i$$
and divide the interval $[0, R]$ into $M$ intervals whose lengths are given by the $r_i$.
3. Generate a uniform random number in the range $[0,R]$ and determine which interval it falls in. 
4. Perform the corresponding transition and update the time variable by an amount $\Delta t = 1/R$.
5. Repeat until the desired simulation time is reached.

To be concrete, suppose with have $N$ particles on the interval $[-L, L]$ initially located at $x=0$. Each particle can hop a distance $\Delta x$ to the left or to the right with equal probability. The rate of hopping is initially considered to be the same for all particles:

$$ r_i = \frac{D}{2}\,\frac{1}{(\Delta x)^2} $$

This choice ensures that in the continous limit ($\Delta x \to 0$, $N\to \infty$), the density of particles $c(x,t)$ on the line will satisfy the usual diffusion equation with diffusion coefficient $D$ (recall MA933):

$$ \frac{\partial c}{\partial t} = D \frac{\partial^2 c}{\partial x^2}$$

with initial condition $c(x,0) = \delta(x)$.

The steps of the algorithm then correspond to the following:
1. There are $M= 2 N$ possible transitions at each step: each of $N$ particles can hop a distance $\Delta x$ either left or right with the above rates. We index them as follows: events k=1:N correspond to particle k hopping to the right, events k=N+1:2N correspond to particle k-N hopping to the left.
2. The total rate is 
$$ R = \frac{D N}{(\Delta x)^2}.$$
3. Generate a uniform random number in the range $[0,R]$ and determine the index $k$ of particle to move and the direction.
4. Update the position of particle $k$ accordingly.
5. Repeat.

The following piece of code implements the Gillespie algorithm for N=100, L=10, $\Delta x = 0.1$ and $D=1$, converts the particle positions to particle density and compares to the solution of the diffusion equation on an unbounded domain at t=1 (we assume that L=10 is far enough from zero to be considered unbounded):

$$c(x, t) = \frac{1}{\sqrt{2 \pi D t}}\, \exp [-\frac{1}{2}\frac{x^2}{D t} ]$$

In [9]:
L  = 10.0
N  = 100
Nx = 201
dx = 2.0*L/(Nx-1) 
X  = dx.*(-(Nx-1)/2:(Nx-1)/2)
Y  = zeros(Int64,N)
D  = 1.0
t  = 0.0


r = (D/2.0)/(dx^2)
totalRate = 2.0*N*r
dt = 1.0/totalRate
T=15.0

# This is the main loop
while t < T
    # Pick an event
    k = rand(1:2*N)
    if k<=N
        hop = 1
        particleId = k
    else
        hop = -1
        particleId=k-N
    end
    Y[particleId]+=hop
    t+=dt
end

# Calculate the estimated density of particles
P = zeros(Float64,length(X))
for i in 1:length(Y)
    P[Y[i]+Int64((Nx-1)/2)+1]+=1/(N * dx)
end

# Calculate the theoretical density and compare
function normal(x, D, t)
    return (1.0/sqrt(2.0*pi*D*t))*exp(-x*x/(2*D*t))
end
P1 = normal.(X, D, T)

plot(X, P1, label="Analytic solution of diffusion equation")
plot!(X, P, label="Numerical estimate of particle density")
xlabel!("x")
ylabel!("Particle density, c(x,t)")
title!("Stochastic simulation of simple diffusion in 1-D")


**1)** The code above makes critical use of the fact that all intervals have the same width in order to decide which particle to move at each step. Suppose now that instead of being equal to $D$ for each particle, the diffusion coefficients are drawn from an exponential distribution with mean $D$:

$$ P(x) = \frac{1}{D} \exp\left(-\frac{x}{D}\right). $$

Modify the code to account for this, using your code from Q2 above to solve the interval membership problem using linear search. You will need to use Julia's randexp() function to generate values for the diffusion coefficients. What is the largest value of $N$ you can simulate in a "reasonable" time?

**2)** Try again using your Fenwick tree from Q2 to solve the interval membership problem and compare. What is the largest value of $N$ you can simulate now?

**3)** Show analytically that the solution of the heterogenous diffusion problem is

$$ c(x,t) = \frac{\exp \left(-\sqrt{\frac{2}{D\,t}}\, |x| \right)}{\sqrt{2\, D\, t}}$$

You will probably need the integral formula (Formula 3.325 of Gradshteyn and Rizhik, 7th Ed.)

$$ \int_0^\infty \exp\left(-a\,x^2 - \frac{b}{x^2} \right)\, dx =  \frac{1}{2}\,\sqrt{\frac{\pi}{a}}\,\exp\left(-2 \sqrt{a\,b} \right).$$
Use this formula to validate your numerical results.



## Using Linear (Brute) Search

In [30]:
L  = 10.0
N  = 100
Nx = 201
dx = 2.0*L/(Nx-1) 
X  = dx.*(-(Nx-1)/2:(Nx-1)/2)
Y  = zeros(Int64,N)
D  = 1.0
t  = 0.0

Drates = randexp(rng, N)
Drates = [Drates;Drates]
rates = Drates/2 * 1/(dx^2)

rate_intervals = buildLList( KVPair.( collect(1:2*N), σ ) )

Σ = sum(rates)
totalRate = Σ
dt = 1.0/totalRate
T=10.0

# This is the main loop
while t < T
    # Pick an event
    rate_selected = rand()*Σ
    k = intervalmembership_brute(rate_intervals,rate_selected).key
    if k<=N
        hop = 1
        particleId = k
    else
        hop = -1
        particleId=k-N
    end
    Y[particleId]+=hop
    t+=dt
end

# Calculate the estimated density of particles
P = zeros(Float64,length(X))
for i in 1:length(Y)
    P[Y[i]+int64((Nx-1)/2)+1]+=1/(N * dx)
end

# Calculate the theoretical density and compare
function normal(x, D, t)
    return (1.0/sqrt(2.0*pi*D*t))*exp(-x*x/(2*D*t))
end
P1 = normal.(X, D, T)

plot(X, P1, label="Analytic solution of diffusion equation")
plot!(X, P, label="Numerical estimate of particle density")
xlabel!("x")
ylabel!("Particle density, c(x,t)")
title!("Stochastic simulation of simple diffusion in 1-D")


LoadError: DimensionMismatch("arrays could not be broadcast to a common size")

## Using Tree Search

In [68]:
L  = 10.0
N  = 1000
Nx = 201
dx = 2.0*L/(Nx-1) 
X  = dx.*(-(Nx-1)/2:(Nx-1)/2)
Y  = zeros(Int64,N)
D  = 1.0
t  = 0.0

# Generates rates related variables
Drates = randexp(N)
Drates = [Drates;Drates]
rates = ( Drates/2 ) * ( 1/(dx^2) )
# σ = cumsum(rates)
# Σ = σ[end]
Σ = sum(rates)

rates_pairs = KVPair.( collect( [1:N;N+1:2*N] ), rates )
rate_tree = Nullable{FTree}(FTree(KVPair(0,0.0)))
rate_tree = buildFTree(rate_tree, rates_pairs);


totalRate = Σ
dt = 1.0/totalRate
T=1.0

# This is the main loop
while t < T
    # Pick an event
    rate_selected = Float64(rand(Uniform(0, Σ)))
    k = intervalmembership(rate_tree, rate_selected)
    k = k.key
    if k<=N
        hop = 1
        particleId = k
    else
        hop = -1
        particleId=k-N
    end
    Y[particleId]+=hop
    t+=dt
end

## BIG BUG
# Assumes the distance of the particles does not exceed a given number (believe it is 201)
# this will be correct for small T, but will fail for higher T

# Calculate the estimated density of particles
P = zeros(Float64,length(X))
for i in 1:length(Y)
    P[Y[i]+Int64((Nx-1)/2)+1]+=1/(N * dx)
end

# # Calculate the theoretical density and compare
function normal(x, D, t)
#     return (1.0/sqrt(2.0*pi*D*t))*exp(-x*x/(2*D*t))
    return exp( -sqrt(2 / (D*t) ) * abs(x) ) / sqrt( 2*D*t )
end

P1 = normal.(X, D, T)

Plots.plot(X, P1, label="Analytical solution of diffusion equation")
plot!(X, P, label="Numerical estimate of particle density")
xlabel!("x")
ylabel!("Particle density, c(x,t)")
title!("Stochastic simulation of simple diffusion in 1-D")


$$ c(x,t) = \frac{\exp \left(-\sqrt{\frac{2}{D\,t}}\, |x| \right)}{\sqrt{2\, D\, t}}$$
